In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

# Muons

## Muon generation

In [2]:
from tomopt.muon import generate_batch

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
from tomopt.muon import MuonBatch

In [5]:
import torch
from torch import Tensor

In [6]:
torch.randn(int(1e8)).pow(2).mean()

tensor(1.0002)

In [7]:
from tomopt.core import X0

In [8]:
def arb_rad_length(*,z:float, lw:Tensor, size:float) -> float:
    rad_length = torch.ones(list((lw/size).long()))*1e5
    if z >= 0.5 and z <= 0.5: rad_length[...] = X0['lead']#X0['beryllium']
#     if z == 0.6 : rad_length[...] = X0['beryllium']
        
    return rad_length

In [9]:
from tomopt.volume import PassiveLayer, DetectorLayer

### Cost functions

In [10]:
import torch.nn.functional as F

In [11]:
def eff_cost(x:Tensor) -> Tensor:
    return torch.expm1(3*F.relu(x))

In [12]:
def res_cost(x:Tensor) -> Tensor:
    return F.relu(x/100)**2

In [21]:
def get_layers():
    layers = []
    lwh = Tensor([1,1,1])
    size = 0.1
    init_eff = 0.5
    init_res = 100000
    pos = 'above'
    for z,d in zip(np.arange(lwh[2],0,-size), [1,1,0,0,0,0,0,0,1,1]):
        if d:
            layers.append(DetectorLayer(pos=pos, init_eff=init_eff, init_res=init_res,
                                        lw=lwh[:2], z=z, size=size, eff_cost_func=eff_cost, res_cost_func=res_cost))
        else:
            pos = 'below'
            layers.append(PassiveLayer(rad_length_func=arb_rad_length, lw=lwh[:2], z=z, size=size))

    return nn.ModuleList(layers) 

In [22]:
import numpy as np
from torch import nn

In [23]:
from tomopt.volume import Volume

In [24]:
batch = MuonBatch(generate_batch(1000), init_z=1)

In [25]:
volume = Volume(get_layers())

In [26]:
volume(batch)

# Atomic number reco

## Compute deviations and scatter locations

In [51]:
from tomopt.inference import ScatterBatch

In [52]:
scatters = ScatterBatch(batch, volume)

## Infer rad length

In [53]:
from tomopt.inference import X0Inferer
from tomopt.core import *

In [54]:
from typing import *

In [311]:
class X0Inferer:
    def __init__(self, scatters: ScatterBatch, default_pred: Optional[float] = X0["beryllium"]):
        self.scatters, self.default_pred = scatters, default_pred
        self.mu, self.volume, self.hits = self.scatters.mu, self.scatters.volume, self.scatters.hits
        self.size, self.lw = self.volume.size, self.volume.lw
        self.mask = self.scatters.get_scatter_mask()

    def x0_from_dtheta(self) -> Tuple[Tensor, Tensor]:
        r"""
        TODO: Debias by considering each voxel on muon paths
        Maybe like:
        Debias dtheta
        dtheta_unc2 = dtheta_unc.pow(2)
        dtheta_dbias = dtheta.pow(2)-dtheta_unc2
        m = [dtheta_dbias < dtheta_unc2]
        dtheta_dbias[m] = dtheta_unc2[m]
        dtheta_dbias = dtheta_dbias.sqrt()
        dtheta = dtheta_dbias
        """

        mom = self.mu.reco_mom[self.mu.get_xy_mask(self.lw)][self.mask]
        dtheta = self.scatters.dtheta[self.mask]
        dtheta_unc = self.scatters.dtheta_unc[self.mask]
        theta_xy_in = self.scatters.theta_in[self.mask]
        theta_xy_out = self.scatters.theta_out[self.mask]
        theta_xy_in_unc = self.scatters.theta_in_unc[self.mask]
        theta_xy_out_unc = self.scatters.theta_out_unc[self.mask]

        # Prediction
        theta2 = dtheta.pow(2).sum(1)
        n_x0 = 0.5 * theta2 * ((mom / SCATTER_COEF_A) ** 2)
        theta_in = theta_xy_in.pow(2).sum(1).sqrt()
        theta_out = theta_xy_out.pow(2).sum(1).sqrt()
        cos_theta_in = torch.cos(theta_in)
        cos_theta_out = torch.cos(theta_out)
        cos_mean = (cos_theta_in + cos_theta_out) / 2
        pred = self.size / (n_x0 * cos_mean)

        # Uncertainty TODO probably best check this
        theta2_unc = (2 * dtheta * dtheta_unc).pow(2).sum(1).sqrt()
        n_x0_unc = 0.5 * theta2_unc * ((mom / SCATTER_COEF_A) ** 2)
        theta_in2_unc = (2 * theta_xy_in * theta_xy_in_unc).pow(2).sum(1).sqrt()
        theta_in_unc = 0.5 * theta_in2_unc / theta_in
        theta_out2_unc = (2 * theta_xy_out * theta_xy_out_unc).pow(2).sum(1).sqrt()
        theta_out_unc = 0.5 * theta_out2_unc / theta_out
        cos_theta_in_unc = torch.sin(theta_in) * theta_in_unc
        cos_theta_out_unc = torch.sin(theta_out) * theta_out_unc
        cos_mean_unc = torch.sqrt(cos_theta_in_unc.pow(2) + cos_theta_out_unc.pow(2)) / 2
        inv_cos_mean_unc = cos_mean_unc / cos_mean.pow(2)
        inv_n_x0_unc = n_x0_unc / n_x0.pow(2)
        pred_unc = pred * torch.sqrt((inv_n_x0_unc * n_x0).pow(2) + (inv_cos_mean_unc * cos_mean).pow(2))

        return pred, pred_unc

    def x0_from_dxy(self) -> Tuple[Optional[Tensor], Optional[Tensor]]:
        # TODO: FIX this
        # dxy = torch.sqrt(scatters['dxy'][mask].pow(2).sum(1))
        # dh = dxy/((math.sqrt(2)*torch.cos(scatters['theta_in'][mask].pow(2).sum(1)))+1e-17)
        # theta0 = torch.arcsin(dh/self.size)
        # x0_pred_dxy = (theta0*p/b)**2
        return None, None

    def compute_efficiency(self) -> Tensor:
        eff = None
        for p, l, i in zip(("above", "above", "below", "below"), self.volume.get_detectors(), (0, 1, 0, 1)):
            x = l.abs2idx(self.hits[p]["xy"][:, i][self.mask])
            e = l.efficiency[x[:, 0], x[:, 1]]
            if eff is None:
                eff = e
            else:
                eff = eff * e
        if eff is None:
            eff = torch.zeros(0)
        return eff

    def average_preds(
        self, x0_dtheta: Optional[Tensor], x0_dtheta_unc: Optional[Tensor], x0_dxy: Optional[Tensor], x0_dxy_unc: Optional[Tensor], efficiency: Tensor
    ) -> Tuple[Tensor, Tensor]:
        r"""
        TODO: Use location uncertainty to spread muon inferrence over neighbouring voxels around central location
        """

        loc, loc_unc = self.scatters.location[self.mask], self.scatters.location_unc[self.mask]  # noqa F841 will use loc_unc to infer around central voxel
        loc_idx = self.volume.lookup_xyz_coords(loc, passive_only=True)
        idxs = torch.stack((torch.arange(len(loc)).long(), loc_idx[:, 2], loc_idx[:, 0], loc_idx[:, 1]), dim=1)
        shp = (len(loc), len(self.volume.get_passives()), *(self.volume.lw / self.volume.size).long())
        
        preds, weights = [], []
        for x0, unc in ((x0_dtheta, x0_dtheta_unc), (x0_dxy, x0_dxy_unc)):
            if x0 is None or unc is None:
                continue
            coef = efficiency * x0 / ((1e-17) + (unc ** 2))
            preds.append(torch.sparse_coo_tensor(idxs.T, x0*coef, size=shp).to_dense())
            weights.append(torch.sparse_coo_tensor(idxs.T, coef, size=shp).to_dense())

        pred, weight = torch.cat(preds, dim=0), torch.cat(weights, dim=0)
        pred, weight = pred.sum(0), weight.sum(0)
        pred = pred / weight
        return pred, weight

    def add_default_pred(self, pred: Tensor, weight: Tensor) -> None:
        if self.default_pred is None:
            return
        m = pred != pred  # mask NaNs
        pred[m] = self.default_pred
        weight[m] = 1 / (self.default_pred ** 2)

    def pred_x0(self) -> Tuple[Tensor, Tensor]:
        x0_dtheta, x0_dtheta_unc = self.x0_from_dtheta()
        x0_dxy, x0_dxy_unc = self.x0_from_dxy()
        eff = self.compute_efficiency()

        pred, weight = self.average_preds(x0_dtheta=x0_dtheta, x0_dtheta_unc=x0_dtheta_unc, x0_dxy=x0_dxy, x0_dxy_unc=x0_dxy_unc, efficiency=eff)
        self.add_default_pred(pred, weight)
        return pred, weight


In [312]:
i = Tensor([[0, 2], [1, 0], [1, 2]]).long()
v =  Tensor([3,      4,      5    ])
s = torch.sparse_coo_tensor(i.T, v, [2,3])

In [313]:
i.T.shape, v.shape

(torch.Size([2, 3]), torch.Size([3]))

In [314]:
list(zip(*i))

[(tensor(0), tensor(1), tensor(1)), (tensor(2), tensor(0), tensor(2))]

In [315]:
i.shape

torch.Size([3, 2])

In [316]:
list(zip(*[[6,1,10]]))

[(6,), (1,), (10,)]

In [317]:
torch.sparse_coo_tensor(list(zip(*[[5,1,9]])), Tensor([2]), size=(6,10,10)).to_dense().shape

torch.Size([6, 10, 10])

In [318]:
x0_inferer = X0Inferer(scatters)

In [319]:
pred_bias, unc = x0_inferer.x0_from_dtheta()

In [320]:
(pred_bias/unc).max()

tensor(1.2472, grad_fn=<MaxBackward1>)

In [321]:
pred,w = x0_inferer.pred_x0()

In [324]:
pred

tensor([[[8.3595e+00, 3.5280e-01, 3.5280e-01, 3.5280e-01, 1.8168e+01,
          5.7232e+01, 6.8062e+01, 3.5280e-01, 3.5280e-01, 3.5280e-01],
         [3.5280e-01, 3.5280e-01, 1.5636e+01, 1.2335e+01, 2.3507e+01,
          3.5280e-01, 1.0596e+01, 3.5280e-01, 1.5747e+01, 3.5280e-01],
         [3.5280e-01, 2.6895e+02, 2.6946e+02, 8.6303e+00, 3.5280e-01,
          1.7416e+01, 1.1203e+01, 1.8570e+01, 1.2051e+01, 2.5010e+01],
         [3.5280e-01, 3.5280e-01, 1.6565e+01, 3.5280e-01, 2.2678e+01,
          7.0060e+00, 3.5280e-01, 3.5280e-01, 3.5280e-01, 3.5280e-01],
         [3.5280e-01, 1.0737e+02, 1.0328e+01, 3.5280e-01, 3.5280e-01,
          3.5280e-01, 3.5280e-01, 4.7909e+01, 1.2788e+01, 3.5280e-01],
         [3.5280e-01, 9.6732e+00, 3.5280e-01, 3.5280e-01, 3.5280e-01,
          9.4080e+00, 3.5280e-01, 2.0922e+01, 3.5280e-01, 2.0812e+01],
         [3.5280e-01, 3.0962e+01, 1.2863e+01, 1.1575e+01, 1.4312e+01,
          1.7881e+01, 3.5280e-01, 3.5280e-01, 3.5280e-01, 3.5280e-01],
         [1.8

pred_debias.mean()

In [266]:
dtheta, dtheta_unc = scatters.dtheta.detach().numpy(), scatters.dtheta_unc.detach().numpy()

In [ ]:
volume.get_detectors()[-1].resolution.mean()

In [ ]:
sns.distplot(dtheta)

In [ ]:
sns.distplot(dtheta_unc)

In [ ]:
sns.distplot(dtheta_unc/dtheta, bins=1000); plt.xlim([0,10])

In [ ]:
sns.distplot(dtheta)

In [ ]:
sns.distplot(dtheta_unc)

In [ ]:
sns.distplot(dtheta_unc/dtheta, bins=1000); plt.xlim([0,10])

In [ ]:
pred_bias.mean()

In [ ]:
pred_bias.mean()

In [ ]:
pred_bias.mean()

In [ ]:
p = pred_debias.detach().numpy()
pb = pred_bias.detach().numpy()

sns.distplot(p, label=f'debiased mean {p.mean():.2f} max {p.max():.2f}', bins=1000)
sns.distplot(pb, label=f'biased mean {pb.mean():.2f} max {pb.max():.2f}', bins=1000)
plt.legend()
plt.xlim([0,0.2])

In [ ]:
pred.mean()

In [ ]:
p = pred.detach().numpy()
sns.displot(p)
plt.xlim([0,0.2])

In [ ]:
X0['lead'],X0['beryllium']

In [ ]:
pred,weight = x0_inferer.pred_x0()

# Inversion test

In [ ]:
from tomopt.core import X0, SCATTER_COEF_A

In [ ]:
import math

In [ ]:
X0['beryllium']

In [ ]:
deltaz = 0.1
theta_in = Tensor([math.pi/4])
x0 = deltaz / (X0['beryllium']*torch.cos(theta_in))
z1 = 1#torch.randn(n, device=self.device)
z2 = 1#torch.randn(n, device=self.device)

print(x0)
theta0 = (SCATTER_COEF_A / 100) * torch.sqrt(x0)  # Ignore due to inversion problems * (1+(SCATTER_COEF_B*torch.log(x0)))
print(theta0)
theta_msc = math.sqrt(2) * z2 * theta0
print(theta_msc)
phi_msc = Tensor([2 * math.pi])

dtheta_x = theta_msc * torch.cos(phi_msc)
dtheta_y = theta_msc * torch.sin(phi_msc)
dtheta = torch.sqrt(dtheta_x.pow(2)+dtheta_y.pow(2))
theta_out = theta_in+dtheta

In [ ]:
dtheta

In [ ]:
theta2 = dtheta.pow(2)
theta02 = theta2/2
print(theta02.sqrt())
n_x0 = theta02 * ((100 / SCATTER_COEF_A) ** 2)
print(n_x0)
cos_theta_in = torch.cos(theta_in)
deltaz/(n_x0 * cos_theta_in)